In [1]:
import pickle
import torch

In [2]:
folder="/data/home/umang/Trajectory_project/GPS-MTM/outputs/test_UA_atlanta_combined_outliers/2025-08-19_08-34-16/test_outputs/random_masking_0.5_testing/GOAL"
pred_pickle_path="{}/predictions_batch_0.pkl".format(folder)
batch_pickle_path = "{}/ground_truth_batch_0.pkl".format(folder)
attention_pickle_path = "{}/attention_masks_batch_0.pkl".format(folder)
masks_pickle_path = "{}/masks_batch_0.pkl".format(folder)
with open(pred_pickle_path, 'rb') as f:
    predictions = pickle.load(f)

with open(batch_pickle_path, 'rb') as f:
    batch = pickle.load(f)

with open(attention_pickle_path, 'rb') as f:
    attention_masks = pickle.load(f)

with open(masks_pickle_path, 'rb') as f:
    masks = pickle.load(f)

In [3]:
masks

{'states': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1

In [4]:
predictions["states"].shape

torch.Size([32, 278, 4])

In [5]:
predictions["actions"].shape

torch.Size([32, 278, 11])

In [6]:
# get the most frequently top-5 most repeated in ground_truth_list_States
# #don't use torch top-k..
ground_truth_list_states = []
for i in range(len(attention_masks)):
     # first zero mask is first time when attention_masks[i] becomes zero
    zero_indices = (attention_masks[i].flatten() == 0).nonzero(as_tuple=True)[0]
    first_zero_mask = zero_indices[0].item() if len(zero_indices) > 0 else attention_masks[i].numel()
    
    ground_truth_states= batch["states"][i, :first_zero_mask, :]
    ground_truth_states = torch.argmax(ground_truth_states, dim=-1)
    ground_truth_list_states.append(ground_truth_states)

k=4
ground_truth_list_states = torch.cat(ground_truth_list_states).flatten()
unique, counts = torch.unique(ground_truth_list_states, return_counts=True)
top_k_ground_truth = unique[torch.topk(counts, k=k).indices]
top_k_ground_truth

top_1_ground_truth = unique[torch.topk(counts, k=1).indices]
top_4_ground_truth = unique[torch.topk(counts, k=4).indices]    
# top_5_ground_truth = unique[torch.topk(counts, k=5).indices]
# top_10_ground_truth = unique[torch.topk(counts, k=10).indices]
# top_20_ground_truth = unique[torch.topk(counts, k=20).indices]


In [7]:
predictions_list_states=[]
predictions_list_actions=[]
ground_truth_list_states=[]
ground_truth_list_actions=[]
correct_list=[]

for i in range(len(attention_masks)):
    # first zero mask is first time when attention_masks[i] becomes zero
    zero_indices = (attention_masks[i].flatten() == 0).nonzero(as_tuple=True)[0]
    first_zero_mask = zero_indices[0].item() if len(zero_indices) > 0 else attention_masks[i].numel()
    
    print("first_zero_mask: ", first_zero_mask)
    predictions_states= predictions["states"][i, :first_zero_mask, :]
    predictions_actions= predictions["actions"][i, :first_zero_mask, :]
    ground_truth_states= batch["states"][i, :first_zero_mask, :]
    ground_truth_actions= batch["actions"][i, :first_zero_mask, :]

    # take argmax of predictions states on last dimension
    predictions_states = torch.argmax(predictions_states, dim=-1)
    ground_truth_states = torch.argmax(ground_truth_states, dim=-1)

    predictions_list_states.append(predictions_states)
    predictions_list_actions.append(predictions_actions)
    ground_truth_list_states.append(ground_truth_states)
    ground_truth_list_actions.append(ground_truth_actions)

    print(ground_truth_states)
    correct = (predictions_states == ground_truth_states).float()
    correct_list.append(correct)


first_zero_mask:  180
tensor([3, 1, 3, 2, 1, 2, 3, 1, 3, 2, 1, 2, 3, 1, 3, 2, 1, 2, 3, 1, 3, 2, 0, 2,
        0, 2, 0, 2, 0, 1, 0, 2, 3, 1, 3, 2, 0, 2, 1, 2, 0, 2, 0, 2, 0, 2, 0, 2,
        0, 2, 0, 2, 0, 2, 0, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 0, 2, 0, 2,
        0, 2, 0, 2, 3, 2, 3, 1, 3, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 0, 2, 0, 2,
        0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1,
        3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 1, 2, 0, 2, 0, 2, 0, 2, 0, 2, 1, 2, 1, 2,
        1, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 0, 2, 0, 2, 0, 2,
        3, 1, 3, 2, 1, 2, 1, 2, 1, 2, 1, 2])
first_zero_mask:  144
tensor([3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 0, 2, 0, 2,
        0, 2, 1, 2, 3, 1, 3, 2, 3, 1, 3, 2, 0, 2, 0, 2, 0, 2, 0, 2, 3, 1, 3, 2,
        3, 1, 3, 2, 3, 1, 3, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2,
        0, 2, 0, 2, 1, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1,
        3, 2, 0, 2, 0, 2, 0, 2,

In [8]:
ground_truth_list_states[12], predictions_list_states[12]

(tensor([3, 1, 3, 2, 3, 1, 3, 2, 0, 2, 0, 1, 0, 2, 0, 2, 3, 1, 3, 2, 1, 2, 3, 1,
         3, 2, 1, 2, 3, 1, 3, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 1, 0, 2,
         0, 2, 0, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2,
         1, 2, 1, 2, 1, 2, 0, 2, 1, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 1,
         0, 2, 0, 2, 3, 1, 3, 2, 3, 1, 3, 2, 1, 2, 3, 1, 3, 2, 1, 2, 3, 1, 3, 2,
         3, 1, 3, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2,
         0, 1, 0, 2, 0, 2, 3, 1, 3, 2, 1, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2,
         1, 2, 3, 1, 3, 2, 1, 2, 1, 2, 1, 2]),
 tensor([2, 2, 3, 2, 3, 3, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2,
         2, 3, 3, 2, 3, 3, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2,
         2, 3, 3, 2, 3, 3, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2,
         1, 2, 1, 2, 1, 2, 0, 2, 1, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 1,
         0, 2, 0, 2, 3, 1, 3, 2, 3, 1, 3, 2, 1, 2, 3, 1, 3, 2,

In [9]:
predictions_list_states[12]-ground_truth_list_states[12]

tensor([-1,  1,  0,  0,  0,  2, -1,  0,  2,  1,  3,  1,  3,  1,  2,  0, -1,  2,
         0,  0,  2,  1, -1,  1, -1,  1,  2,  0,  0,  2, -1,  0,  2,  1,  3,  0,
         3,  1,  2,  0,  2,  1,  3,  0,  3,  2,  2,  0,  2,  1,  3,  0,  0,  2,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0])

In [10]:
predictions_list_states=torch.cat(predictions_list_states).flatten()
ground_truth_list_states=torch.cat(ground_truth_list_states).flatten()
predictions_list_states.shape, ground_truth_list_states.shape

(torch.Size([5127]), torch.Size([5127]))

In [11]:
predictions_list_states[1720:1730], ground_truth_list_states[1720:1730]

(tensor([0, 2, 1, 2, 0, 2, 0, 2, 0, 2]),
 tensor([0, 2, 1, 2, 0, 2, 0, 2, 0, 2]))

In [12]:
# filter predictions and ground truth for each unique class
unique_classes = torch.unique(ground_truth_list_states)
filtered_predictions = {}
filtered_ground_truth = {}
for cls in unique_classes:
    mask = (ground_truth_list_states == cls)
    filtered_predictions[cls.item()] = predictions_list_states[mask]
    filtered_ground_truth[cls.item()] = ground_truth_list_states[mask]

# sort classes by frequency
class_counts = []
for cls in unique_classes:
    count = (ground_truth_list_states == cls).sum().item()
    class_counts.append((cls.item(), count))

# sort by count in descending order
class_counts.sort(key=lambda x: x[1], reverse=True)

# compute and print accuracy, precision, recall, and f1 score for each class sorted by frequency
print(f"{'Class':<5} {'Count':<6} {'Accuracy':<8} {'Precision':<9} {'Recall':<6} {'F1':<6}")
print("-" * 50)

for cls, count in class_counts:
    # Get predictions and ground truth for this class
    cls_predictions = filtered_predictions[cls]
    cls_ground_truth = filtered_ground_truth[cls]
    
    # Accuracy for this class
    accuracy = (cls_predictions == cls_ground_truth).float().mean().item()
    
    # For precision, recall, F1: need to consider this class vs all others
    # True positives: predicted this class and actually this class
    tp = ((predictions_list_states == cls) & (ground_truth_list_states == cls)).sum().item()
    
    # False positives: predicted this class but actually not this class
    fp = ((predictions_list_states == cls) & (ground_truth_list_states != cls)).sum().item()
    
    # False negatives: didn't predict this class but actually this class
    fn = ((predictions_list_states != cls) & (ground_truth_list_states == cls)).sum().item()
    
    # Calculate precision, recall, F1
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    print(f"{cls:<5} {count:<6} {accuracy:<8.4f} {precision:<9.4f} {recall:<6.4f} {f1:<6.4f}")

Class Count  Accuracy Precision Recall F1    
--------------------------------------------------
2     2071   0.8131   0.6941    0.8131 0.7489
3     1078   0.7996   0.5551    0.7996 0.6553
0     1033   0.5431   0.8618    0.5431 0.6663
1     945    0.5259   1.0000    0.5259 0.6893


In [13]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# Convert to numpy for sklearn compatibility
gt_numpy = ground_truth_list_states.cpu().numpy()
pred_numpy = predictions_list_states.cpu().numpy()

# Overall metrics
overall_accuracy = accuracy_score(gt_numpy, pred_numpy)
macro_f1 = f1_score(gt_numpy, pred_numpy, average='macro')
weighted_f1 = f1_score(gt_numpy, pred_numpy, average='weighted')
macro_precision = precision_score(gt_numpy, pred_numpy, average='macro')
weighted_precision = precision_score(gt_numpy, pred_numpy, average='weighted')
macro_recall = recall_score(gt_numpy, pred_numpy, average='macro')
weighted_recall = recall_score(gt_numpy, pred_numpy, average='weighted')

print("Overall Performance Metrics:")
print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Macro F1 Score: {macro_f1:.4f}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Weighted Precision: {weighted_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")
print(f"Weighted Recall: {weighted_recall:.4f}")

print("\nDetailed Classification Report:")
print(classification_report(gt_numpy, pred_numpy))

# Class-wise metrics accounting for imbalance
unique_classes = torch.unique(ground_truth_list_states)
class_counts = [(cls.item(), (ground_truth_list_states == cls).sum().item()) for cls in unique_classes]
class_counts.sort(key=lambda x: x[1], reverse=True)

print(f"\n{'Class':<5} {'Count':<6} {'Freq%':<6} {'Precision':<9} {'Recall':<6} {'F1':<6}")
print("-" * 50)

for cls, count in class_counts:
    freq_pct = (count / len(ground_truth_list_states)) * 100
    
    # Calculate metrics for this specific class
    cls_mask_gt = (gt_numpy == cls)
    cls_mask_pred = (pred_numpy == cls)
    
    tp = ((pred_numpy == cls) & (gt_numpy == cls)).sum()
    fp = ((pred_numpy == cls) & (gt_numpy != cls)).sum()
    fn = ((pred_numpy != cls) & (gt_numpy == cls)).sum()
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    print(f"{cls:<5} {count:<6} {freq_pct:<6.1f} {precision:<9.4f} {recall:<6.4f} {f1:<6.4f}")

Overall Performance Metrics:
Overall Accuracy: 0.7029
Macro F1 Score: 0.6899
Weighted F1 Score: 0.7016
Macro Precision: 0.7777
Weighted Precision: 0.7550
Macro Recall: 0.6704
Weighted Recall: 0.7029

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.54      0.67      1033
           1       1.00      0.53      0.69       945
           2       0.69      0.81      0.75      2071
           3       0.56      0.80      0.66      1078

    accuracy                           0.70      5127
   macro avg       0.78      0.67      0.69      5127
weighted avg       0.76      0.70      0.70      5127


Class Count  Freq%  Precision Recall F1    
--------------------------------------------------
2     2071   40.4   0.6941    0.8131 0.7489
3     1078   21.0   0.5551    0.7996 0.6553
0     1033   20.1   0.8618    0.5431 0.6663
1     945    18.4   1.0000    0.5259 0.6893


In [14]:
# get the number of correct predictions for classes in top_10_ground_truth
#top_k=[top_1_ground_truth, top_5_ground_truth, top_10_ground_truth, top_20_ground_truth]
top_k=[top_1_ground_truth, top_4_ground_truth]
for i in range(len(top_k)):
    filtered_gt = ground_truth_list_states[(ground_truth_list_states.unsqueeze(-1) == top_k[i]).any(dim=-1)]
    filtered_pred = predictions_list_states[(ground_truth_list_states.unsqueeze(-1) == top_k[i]).any(dim=-1)]

    # get the number of correct predictions and accuracy
    correct_predictions = (filtered_pred == filtered_gt).sum()
    accuracy = correct_predictions / filtered_gt.numel()
    print(f"Number of correct predictions in {top_k[i]} ground truth: {correct_predictions.item()}")
    print(f"Accuracy of correct predictions in {top_k[i]} ground truth: {accuracy.item()}")

Number of correct predictions in tensor([2]) ground truth: 1684
Accuracy of correct predictions in tensor([2]) ground truth: 0.8131337761878967
Number of correct predictions in tensor([2, 3, 0, 1]) ground truth: 3604
Accuracy of correct predictions in tensor([2, 3, 0, 1]) ground truth: 0.7029451727867126


### Proof that model is working properly below.

-0.0207 is the index for mask value to be filled in. Corresponding mask is shown below.

In [15]:
# (Pdb) x_sample[0,:,0]
# tensor([-0.9309, -2.4709, -0.9647, -2.5226, -0.9906, -2.6224, -1.0323, -2.6396,
#         -1.0287, -2.5965, -0.0207, -0.0207, -1.0252, -2.4983, -1.0098, -0.0207,
#         -1.0417, -2.3514, -0.0207, -0.0207, -0.0207, -2.3262, -0.9608, -2.4531,
#         -0.9609, -2.5430, -0.0207, -2.4288, -0.0207, -1.5484, -0.9251, -2.4177,
#         -0.0207, -0.0207, -0.8637, -2.3879, -0.8872, -0.0207, -0.9029, -2.3117,
#         -0.9281, -2.3534, -1.0126, -2.4598, -0.9165, -0.0207, -0.8610, -2.4906,
#         -0.0207, -2.5534, -1.0234, -0.8551, -2.4868, -0.8567, -2.4971, -0.9002,
#         -2.4162, -0.0207, -0.0207, -0.9232, -0.0207, -0.9814, -0.0207, -0.0207,
#         -2.3403, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207], device='cuda:0')

In [16]:
# (Pdb) masks['states']
# tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
#         1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
#         0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
#         1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
#         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
#         0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
#         1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
#         1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#         1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
#         1, 1, 0, 1, 1], device='cuda:0')